In [42]:
import matplotlib as mpl
from matplotlib.widgets import RectangleSelector

mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import cv2
import open3d
from intrinsics import Intrinsics
from scipy.ndimage import gaussian_filter1d
from datetime import datetime
from skimage.restoration import inpaint
import time
from tqdm.notebook import tqdm
from utils import pcd_to_depth_map, pcd_to_volume
import math
import scipy
from scipy.spatial.transform import Rotation

In [43]:
def path_to_date(path):
    date_str = path.split("_")[-1].replace(".png", "")
    date_float = float(date_str)
    return datetime.fromtimestamp(date_float)

In [53]:
def apply_bounding_box(pcd, roi_points):
    p1 = roi_points[np.argmin(roi_points[:, 0])]
    p2 = roi_points[np.argmin(roi_points[:, 2])]
    
    # Test 1
    # p1 = p1 + np.array([0, 0, -.23])
    # p2 = p2 + np.array([.5, 1, -2])    
    
    # Test 2
    # p1 = p1 + np.array([-.5, 0, -.4])
    # p2 = p2 + np.array([.5, .2, -1])
    
    # Test 3
    p1 = p1 + np.array([-.5, 0, -.28])
    p2 = p2 + np.array([0, 1, -2])
    
    # Test 7
    # p1 = p1 + np.array([-.5, 0, 0])
    # p2 = p2 + np.array([0, 1, -5])    
    
    # Test 8 
    # p1 = p1 + np.array([-.5, 0, 0])
    # p2 = p2 + np.array([.5, 1, -5])
    
    points_to_filter = np.asarray(pcd.points)
    
    x_bound = (points_to_filter[:, 0] > p1[0]) & (points_to_filter[:, 0] < p2[0])
    z_bound = (points_to_filter[:, 2] < p1[2]) & (points_to_filter[:, 2] > p2[2])
    y_bound = (points_to_filter[:, 1] > p1[1]) & (points_to_filter[:, 1] < p2[1])
    
    return pcd.select_by_index(np.where(x_bound & z_bound & y_bound)[0])

In [45]:
def get_inpaint_mask(roi, depth):    
    temp_mask = np.zeros(depth.shape)
    mask = np.zeros(depth.shape)
    temp_mask[depth == 0] = 1
    mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]] = temp_mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]]
    return mask

In [46]:
fps = 30
test_name = "test3_depth_deep"
scale = 0.00025
paths = glob.glob(os.path.join("koen_mri_room", test_name, "*.png"))
paths.sort(key=lambda p: int(p.split("\\")[-1].split("_")[1]))

depth = np.asanyarray(open3d.io.read_image(paths[0])) * scale
depth = open3d.geometry.Image(depth.astype(np.float32))

In [47]:
plt.imshow(depth, cmap="gray")
plt.show()

In [7]:
# points = []
# 
# def mouse_event(event):
#     global points
#     global rgb_copy
# 
#     points.append([int(event.xdata), int(event.ydata)])
# 
#     if len(points) == 3:
#         plt.close()
# 
# fig = plt.figure(figsize=(10, 10))
# cid = fig.canvas.mpl_connect('button_press_event', mouse_event)
# plt.imshow(depth)
# plt.show()
# 
# depth_copy = np.asarray(depth).copy()
# for p in points:
#     marked_image = cv2.circle(depth_copy, p, 3, (0, 255, 0), 2)
# 
# fig = plt.figure(figsize=(10, 10))
# plt.imshow(marked_image)
# plt.show()

In [13]:
# fig, ax = plt.subplots()
# ax.imshow(depth, cmap="gray")
# roi = None
# 
# def line_select_callback(eclick, erelease):
#     x1, y1 = eclick.xdata, eclick.ydata
#     x2, y2 = erelease.xdata, erelease.ydata
#     global roi
#     roi = [[int(x1), int(y1)], [int(x2), int(y2)]]
#     plt.close()
# 
# rs = RectangleSelector(ax, line_select_callback,
#                        useblit=False, button=[1],
#                        minspanx=5, minspany=5, spancoords='pixels',
#                        interactive=True)
# 
# 
# plt.connect("key_press_event", rs)
# plt.show()

In [48]:
# Test 1
# points = [[332, 745], [586, 733], [528, 520]]

# Test 2
# points = [[205, 462], [367, 459], [330, 331]]

# Test 3
points = [[201, 467], [370, 467], [324, 319]]

# Test 7
# points = [[208, 455], [365, 455], [304, 291]]

# Test 8
# points = [[332, 734], [565, 730], [494, 457]]

In [62]:
npdepth = np.asarray(depth)
point_mask = np.zeros(npdepth.shape)

for p in points:
    point_mask[p[1], p[0]] = npdepth[p[1], p[0]]

markers_o3d = open3d.geometry.Image(point_mask.astype(np.float32))
intr = Intrinsics(width=640, height=480)
# intr = Intrinsics(width=1024, height=768)

m_pcd = open3d.geometry.PointCloud.create_from_depth_image(markers_o3d, intrinsic=intr.get_intrinsics())
m_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
m_pcd.paint_uniform_color([1.0, 0, 0])

points3d = np.asarray(m_pcd.points).copy()

u_1 = points3d[1] - points3d[2]
u_2 = points3d[0] - points3d[2]

n = np.cross(u_1, u_2)
R = Rotation.align_vectors([0, 1, 0], -n)[0].as_matrix()

In [64]:
# inpaint_mask = get_inpaint_mask(roi, npdepth)
# npdepth = inpaint.inpaint_biharmonic(npdepth, inpaint_mask)

ref_pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=open3d.geometry.Image(npdepth.astype(np.float32)), intrinsic=intr.get_intrinsics())
ref_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
ref_pcd.translate(-points3d[2])
ref_pcd.rotate(R, center=(0, 0, 0))
ref_pcd = apply_bounding_box(ref_pcd, np.asarray(m_pcd.points))
cl, ind = ref_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
ref_pcd = ref_pcd.select_by_index(ind)

axes = open3d.geometry.TriangleMesh.create_coordinate_frame()
open3d.visualization.draw_geometries([pcd, axes])

In [76]:
vis = open3d.visualization.Visualizer()
vis.create_window()

geometry = open3d.geometry.PointCloud()
vis.add_geometry(geometry)
vis.add_geometry(axes)

fps=30
img=None

diff = []
prev_time = path_to_date(paths[0])
for i, path in tqdm(enumerate(paths[1100:]), total=len(paths[1100:])):
    # Uncomment for real-time
    curr_time = path_to_date(path)
    time_diff = (curr_time - prev_time).microseconds
    time.sleep(time_diff/1000000)
    prev_time = curr_time
    
    depth = np.asanyarray(open3d.io.read_image(path)) * scale
    # inpaint_mask = get_inpaint_mask(roi, depth)
    # depth = inpaint.inpaint_biharmonic(depth, inpaint_mask)
    depth = open3d.geometry.Image(depth.astype(np.float32))
    
    pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=depth, intrinsic=intr.get_intrinsics())
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    pcd.translate(-points3d[2])
    pcd.rotate(R, center=(0, 0, 0))
    
    pcd = apply_bounding_box(pcd, np.asarray(m_pcd.points))
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    pcd = pcd.select_by_index(ind)
    
    dists = np.asarray(pcd.compute_point_cloud_distance(ref_pcd))
    diff.append(np.mean(dists))

    geometry.points = pcd.points
        
    if i == 0:
        vis.add_geometry(geometry)

    vis.update_geometry(geometry)
    vis.poll_events()
    vis.update_renderer()
    
vis.destroy_window()

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


  0%|          | 0/1182 [00:00<?, ?it/s]

In [74]:
fig = plt.figure()
plt.plot(diff, label='Volume')
# plt.plot(gaussian_filter1d(np.array(diff), sigma=2))
plt.xlabel("difference")
plt.xlabel("frame")
plt.legend()
plt.show()